# Recruit Restaurant Visitor Forecasting Version III

Following:
https://github.com/dongzhang84/Kaggle/blob/master/Restaurant_Visitor_Forecasting/Visitor_forecasting_v2.ipynb


In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

## Load Data

In [2]:
df_ar = pd.read_csv('data/air_reserve.csv')
df_hr = pd.read_csv('data/hpg_reserve.csv')
df_astore = pd.read_csv('data/air_store_info.csv')
df_hstore = pd.read_csv('data/hpg_store_info.csv')
df_storeid = pd.read_csv('data/store_id_relation.csv')
df_av = pd.read_csv('data/air_visit_data.csv')
date_info = pd.read_csv('data/date_info.csv')
df_submission = pd.read_csv('data/sample_submission.csv')

The data description see:

https://github.com/dongzhang84/Kaggle/blob/master/Restaurant_Visitor_Forecasting/Visitor_forecasting_v1.ipynb

In [4]:
df_ar.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [6]:
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [7]:
df_astore.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [8]:
df_hstore.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [9]:
df_storeid.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [10]:
df_av.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [11]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [12]:
df_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


## TODO next

- df_av, dayofweek, year, month
- df_submission, do the same thing
- stores, df_av-like dataframe. Do min, max, mean, median for df_av and merge with stores
- df_hr, merge with df_storeid, do reserve_lag, reserve_visitor_mean
- df_ar, also do reserve_lag, reserve_visitor_mean
- merge df_hr and df_ar with df_av, and mean these two
- add restaurant information

- Similar steps for df_submission

In [13]:
df_av.shape

(252108, 3)

In [14]:
df_av.dtypes

air_store_id    object
visit_date      object
visitors         int64
dtype: object

In [16]:
df_av['visit_date'] = pd.to_datetime(df_av.visit_date)

In [19]:
df_av['dow'] = df_av.visit_date.dt.dayofweek
df_av.head(3)

,air_store_id,visit_date,visitors,dow
0,air_ba937bf13d40fb24,2016-01-13,25,2
1,air_ba937bf13d40fb24,2016-01-14,32,3
2,air_ba937bf13d40fb24,2016-01-15,29,4


In [20]:
df_av['year'] = df_av.visit_date.dt.year
df_av['month'] = df_av.visit_date.dt.month

In [43]:
df_av['date'] = df_av.visit_date.dt.day

In [44]:
df_av.head(3)

,air_store_id,visit_date,visitors,dow,year,month,date
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,15


In [35]:
#(df_av.visit_date - pd.to_datetime('2016-01-13')).dt.days

In [37]:
df_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [38]:
df_submission['air_store_id'] = df_submission.id.apply(lambda x: x[:-11])

In [39]:
df_submission['visit_date']  = df_submission.id.apply(lambda x: x[-10:])
df_submission['visit_date'] = pd.to_datetime(df_submission.visit_date)

In [45]:
df_submission['dow'] = df_submission.visit_date.dt.dayofweek
df_submission['year'] = df_submission.visit_date.dt.year
df_submission['month'] = df_submission.visit_date.dt.month
df_submission['date'] = df_submission.visit_date.dt.day

In [46]:
df_submission.head()

,id,visitors,air_store_id,visit_date,dow,year,month,date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,6,2017,4,23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,0,2017,4,24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,1,2017,4,25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,2,2017,4,26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,3,2017,4,27


In [48]:
# Next to create: stores

unique_stores = df_submission['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], 
                   axis=0, ignore_index=True).reset_index(drop=True)

In [49]:
stores.head()

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0
